# We must incept!

* Makes an inception block as described in the " Going Deeper with Convolutions (2014)" piece by google


Firstly we must import keras

In [3]:
import tensorflow.keras as keras

In our case we are given some hints

* A_prev is the output from the previous layer
* filters is a tuple or list containing F1, F3R, F3,F5R, F5, FPP, respectively:
* F1 is the number of filters in the 1x1 convolution
* F3R is the number of filters in the 1x1 convolution before the 3x3 convolution
* F3 is the number of filters in the 3x3 convolution
* F5R is the number of filters in the 1x1 convolution before the 5x5 convolution
* F5 is the number of filters in the 5x5 convolution
* FPP is the number of filters in the 1x1 convolution after the max pooling
* All convolutions inside the inception block should use a rectified linear activation (ReLU)
* Returns: the concatenated output of the inception block

In [4]:
def inception_block(A_prev, filters):
    """
    Makes the Google inception block

    :param A_prev: input tensor from previous layer

    :param filters: filters is a tuple or list containing
    F1, F3R, F3,F5R, F5, FPP, respectively

    all conv layers must use ReLU activation function

    Returns: the concatenated output of the inception block
"""
    layers = keras.layers
    f1 = filters[0]
    f3r = filters[1]
    f3 = filters[2]
    f5r = filters[3]
    f5 = filters[4]
    fpp = filters[5]

    tower_1 = layers.Conv2D(
            filters=f1,
            kernel_size=(1, 1),
            padding='same',
            activation='relu',
        )

    tower_2 = layers.Conv2D(
            filters=f3r,
            kernel_size=(1, 1),
            padding='same',
            activation='relu',
        )

    tower_3 = layers.Conv2D(
        filters=f3,
        kernel_size=(3, 3),
        padding='same',
        activation='relu',
    )

    tower_4 = layers.Conv2D(
        filters=f5r,
        kernel_size=(1, 1),
        padding='same',
        activation='relu',
    )

    tower_5 = layers.Conv2D(
        filters=f5,
        kernel_size=(5, 5),
        padding='same',
        activation='relu',
    )

    pool = layers.MaxPooling2D((3, 3), strides=(1, 1),
                                  padding='same')(A_prev)

    tower_6 = layers.Conv2D(
        filters=fpp,
        kernel_size=(1, 1),
        padding='same',
        activation='relu',
    )

    layer_1 = tower_1(A_prev)
    layer_3_prev= tower_2(A_prev)
    layer_3 = tower_3(layer_3_prev)
    layer_5_prev= tower_4(A_prev)
    layer_5 = tower_5(layer_5_prev)
    layer_end = tower_6(pool)

    output = layers.concatenate([layer_1, layer_3, layer_5, layer_end])

    return output


#### Lets check out the output

In [5]:
#!/usr/bin/env python3

if __name__ == '__main__':
    X = keras.Input(shape=(224, 224, 3))
    Y = inception_block(X, [64, 96, 128, 16, 32, 32])
    model = keras.models.Model(inputs=X, outputs=Y)
    model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_1 (Conv2D)   │ (None, 224, 224,  │        384 │ input_layer[0][0] │
│                     │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_3 (Conv2D)   │ (None, 224, 224,  │         64 │ input_layer[0][0] │
│                     │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d       │ (None, 224, 224,  │          0 │ input_layer[0][0] │
│ (MaxPooling2D)      │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d (Conv2D)     │ (None, 224, 224,  │        256 │ input_layer[0][0] │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_2 (Conv2D)   │ (None, 224, 224,  │    110,720 │ conv2d_1[0][0]    │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_4 (Conv2D)   │ (None, 224, 224,  │     12,832 │ conv2d_3[0][0]    │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_5 (Conv2D)   │ (None, 224, 224,  │        128 │ max_pooling2d[0]… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 224, 224,  │          0 │ conv2d[0][0],     │
│ (Concatenate)       │ 256)              │            │ conv2d_2[0][0],   │
│                     │                   │            │ conv2d_4[0][0],   │
│                     │                   │            │ conv2d_5[0][0]    │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 124,384 (485.88 KB)

 Trainable params: 124,384 (485.88 KB)

 Non-trainable params: 0 (0.00 B)

### Task 1 be like... PAIN

The second (task 1) task needs us to make the Inception ***Network***, which is just the architecture Google used for their own. This will also include the convolution blocks.

In [18]:
def inception_network():
  """
  Makes the Google Inception Network

  The input shape is (224, 224, 3)

  Returns: The keras model
  """
  dense = keras.layers.Dense
  conv = keras.layers.Conv2D
  max_pool = keras.layers.MaxPooling2D
  inception = inception_block
  avg_pool = keras.layers.AveragePooling2D
  drop = keras.layers.Dropout
  
  input_layer = keras.layers.Input(shape=(224, 224, 3))
  
  x = conv(
      filters=64,
      kernel_size=(7, 7),
      padding='same',
      strides=(2, 2),
      activation='relu',
      name='conv_start',
  )(input_layer)
  
  x = max_pool(
      (3, 3),
      strides=(2, 2),
      padding='same',
      name='max_pool_1',
  )(x)
  
  x = conv(
      filters=64,
      kernel_size=(1, 1),
      padding='same',
      strides=(1, 1),
      activation='relu',
      name="conv_pre_3x3"
  )(x)
  
  x = conv(
      filters=192,
      kernel_size=(3, 3),
      padding='same',
      strides=(1, 1),
      activation='relu',
      name="conv_3x3"
  )(x)
  
  x = max_pool(
      (3, 3),
      strides=(2, 2),
      padding='same',
      name='max_pool_2',
  )(x)
  
  x = inception_block(
      x,
      [64, 96, 128, 16, 32, 32],
  )
  
  x = inception_block(
      x,
      [128, 128, 192, 32, 96, 64],
  )
  
  x = max_pool(
      (3, 3),
      strides=(2, 2),
      padding='same',
      name='max_pool_inception_1',
  )(x)
  
  x = inception_block(
      x,
      [192, 96, 208, 16, 48, 64],
  )
  
  x1 = avg_pool(
      (5, 5),
      strides=3
  )(x)
  
  x1 = conv(
      filters=128,
      kernel_size=(1, 1),
      padding='same',
      activation='relu',
  )(x1)
  
  x1 = keras.layers.Flatten()(x1)
  
  x1 = dense(
      1024,
      activation='relu',
  )(x1)
  
  x1 = drop(0.7)(x1)
  
  x1 = dense(10, activation='softmax', name="aux_output_1")(x1)
  
  
  x = inception_block(
      x,
      [160, 112, 224, 24, 64, 64],
  )
  
  x = inception_block(
      x,
      [128, 128, 256, 24, 64, 64],
  )
  
  x = inception_block(
      x,
      [112, 144, 288, 32, 64, 64],
  )
  
  x2 = avg_pool(
      (5, 5),
      strides=3
  )(x)
  
  x2 = conv(
      filters=128,
      kernel_size=(1, 1),
      padding="same",
      activation="relu"
  )(x2)
  
  x2 = keras.layers.Flatten()(x2)
  
  x2 = dense(
      1024,
      activation="relu"
  )(x2)
  
  x2 = drop(0.7)(x2)
  
  x2 = dense(10, activation='softmax', name="aux_output_2")(x2)
  
  x = inception_block(
      x,
      [256, 160, 320, 32, 128, 128],
  )
  
  x = max_pool(
      (3, 3),
      strides=(2, 2),
      padding='same',
      name='max_pool_inception_2',
  )(x)
  
  x = inception_block(
      x,
      [384, 192, 384, 48, 128, 128],
  )
  
  x = inception_block(
      x,
      [384, 192, 384, 48, 128, 128],
  )
  
  x = keras.layers.GlobalAveragePooling2D(name="global_avg_pool")(x)
  
  x = drop(0,4)(x)
  
  x = dense(10, activation='softmax', name="main_output")(x) 
  
  keras_model = keras.models.Model(inputs=input_layer, outputs=x)
  
  return keras_model


In [19]:
#!/usr/bin/env python3

import tensorflow.keras as K

if __name__ == '__main__':
    model = inception_network()
    model.summary()

Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_6       │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv_start (Conv2D) │ (None, 112, 112,  │      9,472 │ input_layer_6[0]… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pool_1          │ (None, 56, 56,    │          0 │ conv_start[0][0]  │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv_pre_3x3        │ (None, 56, 56,    │      4,160 │ max_pool_1[0][0]  │
│ (Conv2D)            │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv_3x3 (Conv2D)   │ (None, 56, 56,    │    110,784 │ conv_pre_3x3[0][… │
│                     │ 192)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pool_2          │ (None, 28, 28,    │          0 │ conv_3x3[0][0]    │
│ (MaxPooling2D)      │ 192)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_176 (Conv2D) │ (None, 28, 28,    │     18,528 │ max_pool_2[0][0]  │
│                     │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_178 (Conv2D) │ (None, 28, 28,    │      3,088 │ max_pool_2[0][0]  │
│                     │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_28    │ (None, 28, 28,    │          0 │ max_pool_2[0][0]  │
│ (MaxPooling2D)      │ 192)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_175 (Conv2D) │ (None, 28, 28,    │     12,352 │ max_pool_2[0][0]  │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_177 (Conv2D) │ (None, 28, 28,    │    110,720 │ conv2d_176[0][0]  │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_179 (Conv2D) │ (None, 28, 28,    │     12,832 │ conv2d_178[0][0]  │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_180 (Conv2D) │ (None, 28, 28,    │      6,176 │ max_pooling2d_28… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_28      │ (None, 28, 28,    │          0 │ conv2d_175[0][0], │
│ (Concatenate)       │ 256)              │            │ conv2d_177[0][0], │
│                     │                   │            │ conv2d_179[0][0], │
│                     │                   │            │ conv2d_180[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_182 (Conv2D) │ (None, 28, 28,    │     32,896 │ concatenate_28[0… │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_184 (Conv2D) │ (None, 28, 28,    │      8,224 │ concatenate_28[0… │
│                     │ 32)               │            │                 

 Total params: 6,528,810 (24.91 MB)

 Trainable params: 6,528,810 (24.91 MB)

 Non-trainable params: 0 (0.00 B)